# Keras: Speeches corpus

In [33]:
#check if gpu is available
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())

from keras import models
from keras import layers
from keras.utils.np_utils import to_categorical

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from utils import plot_results

['/job:localhost/replica:0/task:0/device:GPU:0']


### constants

In [22]:
test_size = 0.2
val_div = int((corpus.shape[0] * (test_size*2))/2)

### load corpus

In [5]:
corpus = pd.read_csv("../data/corpora/speeches_corpus.csv")

In [6]:
corpus.head()

,author,text,title
0,Angela Merkel,"werte Festversammlung , ich bin heute sehr ger...",Rede von Bundeskanzlerin Angela Merkel anlässl...
1,Angela Merkel,"Sehr geehrter Herr Müller , lieber Herr Minist...",Rede von Bundeskanzlerin Angela Merkel anlässl...
2,Angela Merkel,die Fahrt vom Bundeskanzleramt bis zum Deutsch...,Rede der Bundeskanzlerin zur Eröffnung der Aus...
3,Angela Merkel,ich bin wieder gerne bei Ihnen . Das ist schon...,Rede von Bundeskanzlerin Angela Merkel anlässl...
4,Angela Merkel,Sie hatten ja schon gestern eine beeindruckend...,Rede von Bundeskanzlerin Angela Merkel anlässl...


### Vectorizing

In [26]:
vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(corpus["text"])
labels = LabelEncoder().fit_transform(corpus["author"])
vocab = vectorizer.vocabulary_




X_train, X_test, y_train, y_test = train_test_split(vector, 
                                                    labels, 
                                                    test_size=test_size+test_size,
                                                    stratify=corpus["author"],
                                                    shuffle=True)

In [27]:
X_val = X_test[:val_div]
y_val = y_test[:val_div]

X_test = X_test[val_div:]
y_test = y_test[val_div:]

### One-hot encoding

In [28]:
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

## Classifying

### sequential model

In [29]:
model = models.Sequential()
model.add(layers.Dense(64, activation="relu", input_shape=(len(vocab),)))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(len(np.unique(labels)), activation="softmax"))


model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

### training sequential model

In [31]:
history = model.fit(X_train,
                   y_train,
                   epochs=10,
                   batch_size=16,
                   validation_data=(X_test, y_test),
                   verbose=1)

Train on 78 samples, validate on 26 samples
Epoch 1/10
78/78 [==============================] - 0s 589us/step - loss: 0.0502 - acc: 1.0000 - val_loss: 1.6712 - val_acc: 0.6154
Epoch 2/10
78/78 [==============================] - 0s 414us/step - loss: 0.0298 - acc: 1.0000 - val_loss: 1.6750 - val_acc: 0.5769
Epoch 3/10
78/78 [==============================] - 0s 436us/step - loss: 0.0196 - acc: 1.0000 - val_loss: 1.6908 - val_acc: 0.5385
Epoch 4/10
78/78 [==============================] - 0s 465us/step - loss: 0.0133 - acc: 1.0000 - val_loss: 1.6692 - val_acc: 0.6154
Epoch 5/10
78/78 [==============================] - 0s 376us/step - loss: 0.0093 - acc: 1.0000 - val_loss: 1.6522 - val_acc: 0.6538
Epoch 6/10
78/78 [==============================] - 0s 377us/step - loss: 0.0063 - acc: 1.0000 - val_loss: 1.6609 - val_acc: 0.6154
Epoch 7/10
78/78 [==============================] - 0s 427us/step - loss: 0.0042 - acc: 1.0000 - val_loss: 1.7753 - val_acc: 0.5385
Epoch 8/10
78/78 [==============